In [1]:
import numpy as np

from decimal import Decimal
from typing import Dict

In [2]:
def star_vars(star_centres:np.ndarray, delta_h:float=0.1, parameters=[]) -> np.ndarray:
    '''
    Description:
    ------------
    This function generates ``star_points`` based on [1]_ for each
    sample set (i.e., each row consisting of ``star_centres``).
    ``star_centres`` are the points along which in each direction
    the `star_points` are generated. The resolution of sampling is
    :math:`\Delta h` (``delta_h``). This appraoch is a structured
    sampling straregy; read more in [2]_ and [3]_.
    
    
    Arguments:
    ----------
    :param centres: the 2d array (n, m) containing sample sets
                    ``n`` is the number of sample sets and
                    ``m`` is the number of parameters/factors/
                    variables
    :type centres: np.ndarray
    :param delta_h: sampling resolution, defaults to 0.1
    :type delta_h: float
    
    
    Returns:
    --------
    :return star_points: np.array of star points, each element of this 4d
                         array is a 3d np.array with each 2d array containing
                         star points along each parameter/factor/variable.
    :rtype star_points: np.ndarray
    
    
    References:
    -----------
    .. [1] Razavi, S., Sheikholeslami, R., Gupta, H. V., &
           Haghnegahdar, A. (2019). VARS-TOOL: A toolbox for
           comprehensive, efficient, and robust sensitivity 
           and uncertainty analysis. Environmental modelling
           & software, 112, 95-107.
           doi: 10.1016/j.envsoft.2018.10.005
    .. [2] Razavi, S., & Gupta, H. V. (2016). A new framework
           for comprehensive, robust, and efficient global sensitivity
           analysis: 1. Theory. Water Resources Research, 52(1), 423-439.
           doi: 10.1002/2015WR017558
    .. [3] Razavi, S., & Gupta, H. V. (2016). A new framework
           for comprehensive, robust, and efficient global sensitivity
           analysis: 2. Application. Water Resources Research, 52(1),
           423-439. doi: 10.1002/2015WR017559
    
    
    Contributors:
    -------------
    Razavi, Saman, (2016): algorithm, code in MATLAB (c)
    Gupta, Hoshin, (2016): algorithm, code in MATLAB (c)
    Keshavarz, Kasra, (2021): code in Python 3
    '''

    if star_centres.ndim == 1:
        # star_points
        return _star_sampler_dict(star_centres.reshape(1, star_centers.size), delta_h, parameters)
    
    elif star_centres.ndim == 2:
        # star_points
        return _star_sampler_dict(star_centres, delta_h, parameters)
    
    else:
        # cannot operate on more than 2 dimensional arrays at the moment
        raise ValueError('dimension mismatch: "star_centres" must be a 1- or 2-dimensional array')

        
def _star_sampler_dict(centres, resolution=0.1, parameters=[]):
    '''
    
    '''
    if not parameters:
        parameters = list(range(centres.shape[1]))
    
    dict_of_points = {}
    points = {}

    for i in range(len(centres)):
        row = centres[i,:]
        for k in range(len(row)):
            idx_size = len(_range_vector(row[k], step=resolution))
            col_size = row.size
            
            # a bit high memory usage but nothing else comes to my minds for now (KK)
            temp_view = np.broadcast_to(row, (idx_size, col_size)).reshape(idx_size, col_size).copy()
            temp_view[:,k] = _range_vector(row[k], step=resolution)
            points[parameters[k]] = temp_view
        dict_of_points[i] = points
    
    return dict_of_points


def _range_vector(num, start=0, end=1, step=0.1):
    '''
    Produces the ranges between 0 and 1 with
    incremental steps while including 0<``num``<1.
    '''
    
    first_part = [-i for i in rangef(-num, start, step)]
    second_part = [i for i in rangef(num, end, step)]
    
    return np.unique(np.array(first_part + second_part))


def rangef(start, stop, step, fround=5):
    """
    Yields sequence of numbers from start (inclusive) to stop (inclusive)
    by step (increment) with rounding set to n digits.

    :param start: start of sequence
    :param stop: end of sequence
    :param step: int or float increment (e.g. 1 or 0.001)
    :param fround: float rounding, n decimal places
    :return:
    """
    try:
        i = 0
        while stop >= start and step > 0:
            if i==0:
                yield start
            elif start >= stop:
                yield stop
            elif start < stop:
                if start == 0:
                    yield 0
                if start != 0:
                    yield start
            i += 1
            start += step
            start = round(start, fround)
        else:
            pass
    except TypeError as e:
        yield "type-error({})".format(e)
    else:
        pass

    
def _cli_save(dict_of_points):
    # temp
    import pandas as pd
    
    return pd.concat({key:pd.concat({k:pd.DataFrame(d) for k, d in value.items()}) for key,value in dict_of_points.items()})

In [3]:
centres = np.random.rand(17,4)
# the name of parameters are just to showcase the function
points_test = star_vars(centres, delta_h=0.7, parameters=['TT', 'C0', 'F1', 'Z5'])
# display(points_test)

In [4]:
_cli_save(points_test)

0         1         2         3
0  TT 0  0.344756  0.527821  0.016993  0.554026
   C0 0  0.344756  0.527821  0.016993  0.554026
   F1 0  0.344756  0.527821  0.016993  0.554026
      1  0.344756  0.527821  0.716990  0.554026
   Z5 0  0.344756  0.527821  0.016993  0.554026
...           ...       ...       ...       ...
16 TT 0  0.344756  0.527821  0.016993  0.554026
   C0 0  0.344756  0.527821  0.016993  0.554026
   F1 0  0.344756  0.527821  0.016993  0.554026
      1  0.344756  0.527821  0.716990  0.554026
   Z5 0  0.344756  0.527821  0.016993  0.554026

[85 rows x 4 columns]